In [2]:
import os
import pandas as pd
import numpy as np

# Function to calculate errors
def calculate_peak_error(x, y, peak_idx, window=3):
    """
    Estimate error in x as the average difference between consecutive x-values.
    """
    x_error = np.mean(np.diff(x))
    return x_error

# Function to process a single file and extract lower and upper peaks
def process_file(file_path, split_point=545):
    print(f"\n--- Processing file: {file_path} ---")
    
    # 1. Read the ODS file using the 'odf' engine
    try:
        data = pd.read_excel(file_path, engine='odf')  # engine='odf' requires 'odfpy' installed
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None, None
    
    # 2. Make sure we have the right columns
    if 'X [nm (air)]' not in data.columns or 'Y [Intensity]' not in data.columns:
        print("Error: The required columns 'X [nm (air)]' or 'Y [Intensity]' are missing.")
        print("Available columns:", data.columns)
        return None, None

    # 3. Convert columns to numeric (in case they're strings)
    data["X [nm (air)]"] = pd.to_numeric(data["X [nm (air)]"], errors="coerce")
    data["Y [Intensity]"] = pd.to_numeric(data["Y [Intensity]"], errors="coerce")

    # 4. Drop rows with NaN in X or Y
    data = data.dropna(subset=["X [nm (air)]", "Y [Intensity]"])

    # 5. Extract x and y arrays
    x = data["X [nm (air)]"]
    y = data["Y [Intensity]"]

    # 6. Split the data at the split point
    lower_data = data[x < split_point]
    upper_data = data[x >= split_point]

    # 7. Extract the angle from the filename
    try:
        angle_str = file_path.split('degree=')[1].split('.')[0]
        angle = int(angle_str)
    except Exception as e:
        print(f"Could not parse angle from filename '{file_path}': {e}")
        angle = None  # or 0, or any default

    # 8. Find peaks in lower_data and upper_data
    lower_peak = None
    if not lower_data.empty:
        lower_peak_idx = lower_data['Y [Intensity]'].idxmax()
        lower_peak = {
            "Angle": angle,
            "x": x[lower_peak_idx],
            "x_error": calculate_peak_error(x.values, y.values, lower_peak_idx)
        }

    upper_peak = None
    if not upper_data.empty:
        upper_peak_idx = upper_data['Y [Intensity]'].idxmax()
        upper_peak = {
            "Angle": angle,
            "x": x[upper_peak_idx],
            "x_error": calculate_peak_error(x.values, y.values, upper_peak_idx)
        }

    return lower_peak, upper_peak

# Main function to process all files in a folder
def process_folder(folder_path, output_file, split_point=545):
    lower_results = []
    upper_results = []
    
    # Loop through all files in the folder
    for file_name in os.listdir(folder_path):
        # Only process .ods files
        if file_name.endswith('.ods'):
            file_path = os.path.join(folder_path, file_name)
            
            # Process each file and capture the peaks
            lower_peak, upper_peak = process_file(file_path, split_point=split_point)
            
            # Append results if available
            if lower_peak is not None:
                lower_results.append(lower_peak)
            if upper_peak is not None:
                upper_results.append(upper_peak)
    
    print(lower_results)
    # Create DataFrames for lower and upper peaks
    lower_df = pd.DataFrame(lower_results)
    upper_df = pd.DataFrame(upper_results)
    
    # Save to Excel
    with pd.ExcelWriter(output_file) as writer:
        lower_df.to_excel(writer, sheet_name='Lower Peaks', index=False)
        upper_df.to_excel(writer, sheet_name='Upper Peaks', index=False)
    
    print(f"\nSummary saved to {output_file}")

# --- Usage Example ---
folder_path = r"C:\Users\David1\Downloads\Exciton-Politron"
output_file = r"C:\Users\David1\Downloads\Exciton-Politron results.xlsx"

process_folder(folder_path, output_file)



--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-10.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-12.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-14.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-16.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-18.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-2.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-20.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-4.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-6.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=-8.ods ---

--- Processing file: C:\Users\David1\Downloads\Exciton-Politron\int=600 degree=0.od